# Data Cleaning and Preparation

**Sources**:

This notebook is a modification of chapter 7 in Python for Data Analysis, 3E. https://wesmckinney.com/book/

All the notebooks of this book are available at: https://github.com/wesm/pydata-book/tree/3rd-edition

**If you are a Colab User**

If you use Colab Notebook, you can uncomment the following cell (by deleting the pound sign in front of each line) <br>
to mount your Google Drive to Colab. After that, your colab notebook can read/write files and data in your Google Drive <br>

please change the current directory to be the folder that you save your Notebook and data folder. <br>For example, I save my Colab files and data in the following location

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

#%cd /content/drive/MyDrive/Colab\ Notebooks

**Import libraries and set up standards for the remainder of the notebook**

In [2]:
# import required libraries and modules, and define default setting for the notebook

import numpy as np
np.random.seed(12345)

import pandas as pd # https://pandas.pydata.org/  Check the documentation there
from pandas import Series, DataFrame # import modules into the local namespace if they are frequently used

import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
pd.options.display.max_columns = 20
pd.options.display.max_colwidth = 80
np.set_printoptions(precision=4, suppress=True)

# Display all outputs from each cell. Otherwise, only the last output is displayed
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


During the course of doing data analysis and modeling, a significant amount of time
is spent on data preparation: loading, cleaning, transforming, and rearranging. Such
tasks are often reported to take up 80% or more of an analyst’s time. Sometimes the
way that data is stored in files or databases is not in the right format for a particular
task. Many researchers choose to do ad hoc processing of data from one form to
another using a general-purpose programming language, like Python, Perl, R, or Java,
or Unix text-processing tools like sed or awk. Fortunately, pandas, along with the
built-in Python language features, provides you with a high-level, flexible, and fast set
of tools to enable us to manipulate data into the right form.

In this module, we study tools for missing data, duplicate data, string manipulation,
and some other analytical data transformations.

##  Handling Missing Data

Missing data occurs commonly in many data analysis applications. One of the goals
of pandas is to make working with missing data as painless as possible. For example,
all of the descriptive statistics on pandas objects exclude missing data by default.
The way that missing data is represented in pandas objects is somewhat imperfect,
but it is sufficient for most real-world use. 

Table 7-1. NA handling object methods


In [3]:
# For data with float64 dtype, pandas uses the floating-point value NaN (Not a Number) 
# to represent missing data.
# We call this a sentinel value: when present, it indicates a missing (or null) value:
# .nan

float_data = pd.Series([1.2, -3.5, np.nan, 0])
float_data

0    1.2
1   -3.5
2    NaN
3    0.0
dtype: float64

In [4]:
# The isna method gives us a Boolean Series with True where values are null

float_data.isna()

# the returning Boolean series indicates the element indexed 2 is null

0    False
1    False
2     True
3    False
dtype: bool

In [5]:
# The built-in Python None value is also treated as NA

string_data = pd.Series(["aardvark", np.nan, None, "avocado"])
string_data
string_data.isna()

float_data = pd.Series([1, 2, None], dtype='float64')
float_data
float_data.isna()

0    aardvark
1         NaN
2        None
3     avocado
dtype: object

0    False
1     True
2     True
3    False
dtype: bool

0    1.0
1    2.0
2    NaN
dtype: float64

0    False
1    False
2     True
dtype: bool

In [6]:
# dropna is a method to drop NaN

data = pd.Series([1, np.nan, 3.5, np.nan, 7])
data

data.dropna()

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

0    1.0
2    3.5
4    7.0
dtype: float64

### Filtering Out Missing Data

notna() method returns a Boolean series for filtering out missing data <br>
dropna(axis, how, thresh) method directly filters out missing data

In [7]:
# the notna method select elements that are not NaN
# .notna() returns a Boolean series, which is used to filtering out entries with missing values
# 
data[data.notna()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [8]:
# There are a few ways to filter out missing data. While we always have the option to
# do it by hand using pandas.isna and Boolean indexing, dropna can be helpful. 
# On a Series, it returns the Series with only the nonnull data and index values

#With DataFrame objects, there are different ways to remove missing data. We may
# want to drop rows or columns that are all NA, or only those rows or columns
# containing any NAs at all. dropna by default drops any row containing a missing
# value

data = pd.DataFrame([[1., 6.5, 3.], [1., np.nan, np.nan],
                     [np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]])
data

data.dropna()

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


,0,1,2
0,1.0,6.5,3.0


In [9]:
# Passing how="all" will drop only rows that are all NaN:

data.dropna(how="all")
# In tis example, only row 2 is filter out because it is  the only row containing all NaN

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [10]:
# To drop columns in the same way, pass axis="columns":

# In this example, we add a new empty column labeled as 4
data[4] = np.nan
data

# when axis="columns" or 1, and how="all", columns with all NaN will be dropped
data.dropna(axis="columns", how="all")
#data.dropna(axis=1, how="all")

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [11]:
# Suppose we want to keep only rows containing at most a certain number of missing
# observations. We can indicate this with the thresh argument:

df = pd.DataFrame(np.random.standard_normal((7, 3)))
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan
df

df.dropna(thresh=2) # setting the argument thresh=2 means to drop rows with at least two NaN

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


,0,1,2
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


### Filling in Missing Data

Table 7-2. fillna function arguments

fillna() function fills NaN with the number given
ffill(limit)

In [12]:
# Rather than filtering out missing data (and potentially discarding other data along
# with it), we may want to fill in the “holes” in any number of ways. For most
# purposes, the fillna method is the workhorse function to use. Calling fillna with a
# constant replaces missing values with that value:

df

df.fillna(0)

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [13]:
# Calling fillna with a dictionary,we can use a different fill value for each column
# in this example, we fill NaN in column 1 with 0.5 and in column 2 with 0
df

df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


,0,1,2
0,-0.204708,0.500000,0.000000
1,-0.555730,0.500000,0.000000
2,0.092908,0.500000,0.769023
3,1.246435,0.500000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [14]:
df = pd.DataFrame(np.random.standard_normal((6, 3)))
df.iloc[2:, 1] = np.nan
df.iloc[4:, 2] = np.nan
df

# ffill() method fills missing values forwarded with the value in the last low.
df.ffill()

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,NaN,1.343810
3,-0.713544,NaN,-2.370232
4,-1.860761,NaN,NaN
5,-1.265934,NaN,NaN


,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.124121,1.343810
3,-0.713544,0.124121,-2.370232
4,-1.860761,0.124121,-2.370232
5,-1.265934,0.124121,-2.370232


In [15]:
# the argument limit for the method ffill sets the maximum number of successive missing values to be 
# filled using the forward filling method
# In this example, limit=2 means the maximum number of successive missing values to be filled is 2

df.ffill(limit=2)

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.124121,1.343810
3,-0.713544,0.124121,-2.370232
4,-1.860761,NaN,-2.370232
5,-1.265934,NaN,-2.370232


In [16]:
# fill missing data with column means

data = pd.Series([1., np.nan, 3.5, np.nan, 7])
print(data,'\n')

data.fillna(data.mean())

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64 



0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## Data Transformation

Filtering, cleaning, and other transformations are another class of important operations

### Removing Duplicates

duplicated() method returns boolean Series denoting duplicate rows <br>
dropduplicates() returns DataFrame with duplicate rows removed.

In [17]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                     "k2": [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [18]:
# It is important to know if a dataset has duplicated entries. 
# The method duplicated() returns a Boolean Series indicating whether each row is a duplicate

data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [19]:
# the drop_duplicates method returns a DataFrame with rows where the duplicated array is True filtered out

data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [20]:
# we can specify any subset of them to detect duplicates. Suppose we had an additional column of
# values and wanted to filter duplicates based only on the "k1" column:

data["v1"] = range(7)
data

data.drop_duplicates(subset=["k1"]) 
# In this example, only column "k1" is considered when determining duplications.
# "k1" in rows 2 to 6 have duplicated values "one" and "two". Therefore, rows 2-6 are all dropped

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


,k1,k2,v1
0,one,1,0
1,two,1,1


In [21]:
# duplicated and drop_duplicates by default keep the first observed value combination.
# Passing keep="last" will return the last one:
data

# on the subset comprising columns "k1" and "k2" are considered when assessing duplicated values.
# when passing "last" to the argument keep, searching duplicated rows is backward, from the bottom to the top 
data.duplicated(["k1", "k2"],keep="last") 

# row 5 and 6 have the same values for the considered subset.
# When the argument keep="last", row 6 is kept and row 5 is dropped
data.drop_duplicates(["k1", "k2"], keep="last")

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


0    False
1    False
2    False
3    False
4    False
5     True
6    False
dtype: bool

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping

For many datasets, you may wish to perform some transformation based on the values in an array, Series, or column in a DataFrame.

In [22]:
# Consider the following hypothetical data collected about various kinds of wastes

#data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon",
#                              "pastrami", "corned beef", "bacon",
#                              "pastrami", "honey ham", "nova lox"],
#                     "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
#data


data = pd.DataFrame({"waste": ["book", "cloth", "water bottle",
                              "jam jar", "package box", "mail",
                              "soda can", "juice bottle", "yard waste"],
                     "weight": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,waste,weight
0,book,4.0
1,cloth,3.0
2,water bottle,12.0
3,jam jar,6.0
4,package box,7.5
5,mail,8.0
6,soda can,3.0
7,juice bottle,5.0
8,yard waste,6.0


In [23]:
# Suppose we wanted to add a column indicating the corresponding material

# Let’s write down a mapping of each distinct waste to the corresponding material

waste_to_material = {
    "book": "paper",
    "cloth": "fabric",
    "water bottle": "plastic",
    "jam jar": "glass",
    "package box": "paper",
    "mail": "paper",
    "soda can": "aluminum",
    "juice bottle": "plastic",
    "yard waste": "organic"
}

In [24]:
# create a new column called "material" whose values are the material for each type of waste
# The map method on a Series accepts a function or dictionary-like object containing a mapping to do
# the transformation of values

data["material"] = data["waste"].map(waste_to_material)
data

,waste,weight,material
0,book,4.0,paper
1,cloth,3.0,fabric
2,water bottle,12.0,plastic
3,jam jar,6.0,glass
4,package box,7.5,paper
5,mail,8.0,paper
6,soda can,3.0,aluminum
7,juice bottle,5.0,plastic
8,yard waste,6.0,organic


### Replacing Values

repace() method can be used to data replacement

In [25]:
# The -999 values might be sentinel values for missing data. 

data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [26]:
# To replace these with NaN values that pandas understands, we can use replace, producing a new Series

data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [27]:
# If we want to replace multiple values at once, we can pass a list and then the substitute value:
data 

data.replace([-999, -1000], np.nan)

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [28]:
#  To use a different replacement for each value, pass a list of substitutes
data

# in this example, we would like to replace -999 with no.nan, and -1000 with 0
data.replace([-999, -1000], [np.nan, 0])

#  The argument passed can also be a dictionary
data.replace({-999: np.nan, -1000: 0})

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes

< This can be skipped >

Like values in a Series, axis labels can be similarly transformed by a function or
mapping of some form to produce new, differently labeled objects. We can also
modify the axes in place without creating a new data structure.

In [29]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=["Ohio", "Colorado", "New York"],
                    columns=["one", "two", "three", "four"])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [30]:
# in this example, we rename axis indexes as the captalized first four characters

def transform(x):
    return x[:4].upper()

data.index.map(transform)

# assign to the index attribute, modifying the DataFrame in place
data.index = data.index.map(transform)
data

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [31]:
# If we want to create a transformed version of a dataset without modifying the
# original, a useful method is rename:
data

data.rename(index=str.title, columns=str.upper)
# here, str.title converts first character of each word to uppercase and remaining to lowercase
# str.upper converts all characers to uppercase

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [32]:
# rename can be used in conjunction with a dictionary-like object, providing 
# new values for a subset of the axis labels

data.rename(index={"OHIO": "INDIANA"},
            columns={"three": "Hold"})

,one,two,Hold,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### Discretization and Binning

Continuous data is often discretized or otherwise separated into “bins” for analysis.<br>
the cut function discretizes a range of data into bins <br>
qcut function discretizes a range of data into bins based on quartiles

In [33]:
# this list saves ages of a group of structures

ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [34]:
# Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older. 
# To do so, we can use pandas.cut function

bins = [18, 25, 35, 60, 100] # bins is a list of bin boundaries in sequence
age_categories = pd.cut(ages, bins)
age_categories # an Categorical object

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [35]:
# attribute categories returns the bins
age_categories.categories
# In this example, the first bin is from 18 to 25. The left boundary is open and right boundary is closed

# attribute codes returns the bin labels of the data 
age_categories.codes 
#In this example, 20 is in bin 0, 22 is in bin 0, ..., 61 is in bin 3, ... 32 is in bin 1

# use value_counts method to find the frequency distribution
age_categories.value_counts()

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
Name: count, dtype: int64

In [36]:
# In the string representation of an interval, a parenthesis means that the side is open
# (exclusive), while the square bracket means it is closed (inclusive). We can change
# which side is closed by passing right=False

pd.cut(ages, bins, right=False)

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, interval[int64, left]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

In [37]:
# We can override the default interval-based bin labeling by passing a list or array to the labels option

group_names = ["Yong", "Middle", "Aging", "Old"]
pd.cut(ages, bins, labels=group_names)

['Yong', 'Yong', 'Yong', 'Middle', 'Yong', ..., 'Middle', 'Old', 'Aging', 'Aging', 'Middle']
Length: 12
Categories (4, object): ['Yong' < 'Middle' < 'Aging' < 'Old']

In [38]:
# If we pass an integer number of bins to pandas.cut instead of explicit bin edges, it
# will compute equal-length bins based on the minimum and maximum values in the
# data. Consider the case of some uniformly distributed data chopped into fourths

data = np.random.uniform(size=20)
pd.cut(data, 4, precision=2)

[(0.34, 0.55], (0.34, 0.55], (0.76, 0.97], (0.76, 0.97], (0.34, 0.55], ..., (0.34, 0.55], (0.34, 0.55], (0.55, 0.76], (0.34, 0.55], (0.12, 0.34]]
Length: 20
Categories (4, interval[float64, right]): [(0.12, 0.34] < (0.34, 0.55] < (0.55, 0.76] < (0.76, 0.97]]

In [39]:
# The function qcut bins the data based on sample quantiles.
# Depending on the distribution of the data, using pandas.cut will not usually result in each bin 
# having the same number of data points. 
# Since pandas.qcut uses sample quantiles instead, we will obtain roughly equally-sized bins:

data = np.random.standard_normal(1000)
quartiles = pd.qcut(data, 4, precision=2)

quartiles

quartiles.value_counts()

[(-0.026, 0.62], (0.62, 3.93], (-0.68, -0.026], (0.62, 3.93], (-0.026, 0.62], ..., (-0.68, -0.026], (-0.68, -0.026], (-2.96, -0.68], (0.62, 3.93], (-0.68, -0.026]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.96, -0.68] < (-0.68, -0.026] < (-0.026, 0.62] < (0.62, 3.93]]

(-2.96, -0.68]     250
(-0.68, -0.026]    250
(-0.026, 0.62]     250
(0.62, 3.93]       250
Name: count, dtype: int64

In [40]:
# Similar to the cut function, we can pass our own quantiles (numbers between 0 and 1, inclusive):

pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]).value_counts()

(-2.9499999999999997, -1.187]    100
(-1.187, -0.0265]                400
(-0.0265, 1.286]                 400
(1.286, 3.928]                   100
Name: count, dtype: int64

### Detecting and Filtering Outliers

Filtering or transforming outliers is largely a matter of applying array operations

Boolean series is commonly used for this purpose

In [41]:
# a DataFrame of 1000 rows x 4 columns
data = pd.DataFrame(np.random.standard_normal((1000, 4)))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.049091,0.026112,-0.002544,-0.051827
std,0.996947,1.007458,0.995232,0.998311
min,-3.645860,-3.184377,-3.745356,-3.428254
25%,-0.599807,-0.612162,-0.687373,-0.747478
50%,0.047101,-0.013609,-0.022158,-0.088274
75%,0.756646,0.695298,0.699046,0.623331
max,2.653656,3.525865,2.735527,3.366626


In [42]:
# find values of column 2 that exceed 3 or -3

data[2].loc[data[2].abs()>3]
# In this example, column 2 is found to have two values exceeding 3 or -3. They are in rows 520 and 897

41    -3.399312
136   -3.745356
Name: 2, dtype: float64

In [43]:
# To select all rows having a value exceeding 3 or –3, you can use the any method on a Boolean DataFrame:

# here, data.abs()>3 returns a Boolean array of 1000x4. (data.abs()>3).any(axis='columns') returns 
# a Series of Boolean values

data[(data.abs() > 3).any(axis="columns")]

,0,1,2,3
41,0.457246,-0.025907,-3.399312,-0.974657
60,1.951312,3.260383,0.963301,1.201206
136,0.508391,-0.196713,-3.745356,-1.520113
235,-0.242459,-3.056990,1.918403,-0.578828
258,0.682841,0.326045,0.425384,-3.428254
322,1.179227,-3.184377,1.369891,-1.074833
544,-3.548824,1.553205,-2.186301,1.277104
635,-0.578093,0.193299,1.397822,3.366626
782,-0.207434,3.525865,0.283070,0.544635
803,-3.645860,0.255475,-0.549574,-1.907459


In [44]:
# limit data within 3/-3 by assigning 3 to upper outliers and -3 to lower outliers

data[data.abs() > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.050286,0.025567,-0.001399,-0.051765
std,0.992920,1.004214,0.991414,0.995761
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.599807,-0.612162,-0.687373,-0.747478
50%,0.047101,-0.013609,-0.022158,-0.088274
75%,0.756646,0.695298,0.699046,0.623331
max,2.653656,3.000000,2.735527,3.000000


### Permutation and Random Sampling

Combination of the permutation method and the take method provides an approaching to permutate or sample rows or columns from a dataset <br>
The sample method randomly samples 

In [45]:
df = pd.DataFrame(np.arange(5 * 7).reshape((5, 7)))
df

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27
4,28,29,30,31,32,33,34


In [46]:
# Permuting (randomly reordering) a Series or the rows in a DataFrame is possible using the 
# numpy.random.permutation function

# Calling permutation with the length of the axis we want to permute produces an array of integers 
# indicating the new ordering

sampler = np.random.permutation(5)
sampler

array([3, 1, 4, 2, 0])

In [47]:
# the take method returns elements from array along the mentioned axis and indices.
# the default axis is 0

df.take(sampler) 

,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27
1,7,8,9,10,11,12,13
4,28,29,30,31,32,33,34
2,14,15,16,17,18,19,20
0,0,1,2,3,4,5,6


In [48]:
# iloc-based indexing method is another approach to permuting rows according to the sampled sequence

df.iloc[sampler]

,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27
1,7,8,9,10,11,12,13
4,28,29,30,31,32,33,34
2,14,15,16,17,18,19,20
0,0,1,2,3,4,5,6


In [49]:
# permutate along columns by setting argument axis="columns"

column_sampler = np.random.permutation(7)
column_sampler
df.take(column_sampler, axis="columns")

array([4, 6, 3, 2, 1, 0, 5])

,4,6,3,2,1,0,5
0,4,6,3,2,1,0,5
1,11,13,10,9,8,7,12
2,18,20,17,16,15,14,19
3,25,27,24,23,22,21,26
4,32,34,31,30,29,28,33


In [50]:
# To select a random subset without replacement (the same row cannot appear twice),
 #we can use the sample method on Series and DataFrame

df.sample(n=3)

,0,1,2,3,4,5,6
2,14,15,16,17,18,19,20
4,28,29,30,31,32,33,34
0,0,1,2,3,4,5,6


In [51]:
# To generate a sample with replacement (to allow repeat choices), pass replace=True to sample

choices = pd.Series([5, 7, -1, 6, 4])
print(choices)

choices.sample(n=10, replace=True)

0    5
1    7
2   -1
3    6
4    4
dtype: int64


2   -1
0    5
3    6
1    7
4    4
0    5
4    4
0    5
4    4
4    4
dtype: int64

In [52]:
# Computing Indicator/Dummy Variables

df = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                   "data1": range(6)})
df

# the get_dummies method generates dummy variables for categorical variable.
pd.get_dummies(df["key"], dtype=float)
# In this example, a dummy varible is generated for the categorical variable "key":
# {"a": [1,0,0], "b": [0,1,0], "c": [0,0,1]

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


,a,b,c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


In [53]:
# if we have multiple categorical variables that should be converted to their dummies, 
# it is helpful to add a prefix to differentiate dummies of different variables

dummies = pd.get_dummies(df["key"], prefix="key", dtype=float)
df_with_dummy = df[["data1"]].join(dummies)
df_with_dummy

# here "key_a", "key_b", "key_c" mean dummies for the categorical variable "key"

,data1,key_a,key_b,key_c
0,0,0.0,1.0,0.0
1,1,0.0,1.0,0.0
2,2,1.0,0.0,0.0
3,3,0.0,0.0,1.0
4,4,1.0,0.0,0.0
5,5,0.0,1.0,0.0


In [54]:
# A useful recipe for statistical applications is to combine get_dummies with a discretization function like cut

np.random.seed(12345) # to make the example repeatable
values = np.random.uniform(size=10) # generate 10 random values from the continuous uniform distribution
values

bins = [0, 0.2, 0.4, 0.6, 0.8, 1] # discretize the numbers into bins
pd.cut(values, bins)


pd.get_dummies(pd.cut(values, bins)) # get dummies for each data according to their bin labels

array([0.9296, 0.3164, 0.1839, 0.2046, 0.5677, 0.5955, 0.9645, 0.6532,
       0.7489, 0.6536])

[(0.8, 1.0], (0.2, 0.4], (0.0, 0.2], (0.2, 0.4], (0.4, 0.6], (0.4, 0.6], (0.8, 1.0], (0.6, 0.8], (0.6, 0.8], (0.6, 0.8]]
Categories (5, interval[float64, right]): [(0.0, 0.2] < (0.2, 0.4] < (0.4, 0.6] < (0.6, 0.8] < (0.8, 1.0]]

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,False,False,False,False,True
1,False,True,False,False,False
2,True,False,False,False,False
3,False,True,False,False,False
4,False,False,True,False,False
5,False,False,True,False,False
6,False,False,False,False,True
7,False,False,False,True,False
8,False,False,False,True,False
9,False,False,False,True,False


## Extension Data Types

< Skip it now >

pandas has developed an extension type system allowing for new data
types to be added even if they are not supported natively by NumPy. These new data
types can be treated as first class alongside data coming from NumPy arrays

In [55]:
s = pd.Series([1, 2, 3, None])
s
s.dtype

0    1.0
1    2.0
2    3.0
3    NaN
dtype: float64

dtype('float64')

In [56]:
s = pd.Series([1, 2, 3, None], dtype=pd.Int64Dtype())
s
s.isna()
s.dtype

0       1
1       2
2       3
3    <NA>
dtype: Int64

0    False
1    False
2    False
3     True
dtype: bool

Int64Dtype()

In [57]:
s[3]
s[3] is pd.NA

<NA>

True

In [58]:
s = pd.Series([1, 2, 3, None], dtype="Int64")

In [59]:
s = pd.Series(['one', 'two', None, 'three'], dtype=pd.StringDtype())
s

0      one
1      two
2     <NA>
3    three
dtype: string

In [60]:
df = pd.DataFrame({"A": [1, 2, None, 4],
                   "B": ["one", "two", "three", None],
                   "C": [False, None, False, True]})
df
df["A"] = df["A"].astype("Int64")
df["B"] = df["B"].astype("string")
df["C"] = df["C"].astype("boolean")
df

,A,B,C
0,1.0,one,False
1,2.0,two,None
2,NaN,three,False
3,4.0,None,True


,A,B,C
0,1,one,False
1,2,two,<NA>
2,<NA>,three,False
3,4,<NA>,True


## String Manipulation

<skip now because we don't process string data in this class. If you are handling such data for your project, you should read it>

Python has long been a popular raw data manipulation language in part due to its
ease of use for string and text processing. Most text operations are made simple
with the string object’s built-in methods. For more complex pattern matching and
text manipulations, regular expressions may be needed. pandas adds to the mix by
enabling us to apply string and regular expressions concisely on whole arrays of
data, additionally handling the annoyance of missing data.

Table 7-4 a listing of some of Python’s string methods.

### Python Built-In String Object Methods

In [61]:
# the split method can extract substrings separated by a specified symbol:

val = "a,b,  guido"
val.split(",")

['a', 'b', '  guido']

In [62]:
# the split method is often combined with the strip method to trim whitespace (including line breaks)
# strip is a string operation that is used to remove specific characters from both the beginning and 
# the end of a string

pieces = [x.strip() for x in val.split(",")]
pieces

['a', 'b', 'guido']

In [63]:
# These substrings could be concatenated together with a two-colon delimiter using addition:

first, second, third = pieces
first + "::" + second + "::" + third

'a::b::guido'

In [64]:
# A faster way is to pass a list or tuple to the join method on the string "::":

"::".join(pieces)

'a::b::guido'

In [65]:
# Using Python’s in keyword is the best way to detect a substring, though index and find can also be used

"guido" in val

True

In [66]:
val.index(",") # "," is character 1 in the string val

1

In [67]:
# uncomment this example to test. The index method raises an exception if the string isn't found

# val.index(":")

In [68]:
# the find method returns -1 if the string is not found
val.find(":")

# Note that the difference between find and index is that index raises an exception if the string 
# isn’t found (versus returning –1)

-1

In [69]:
# the count method returns the number of occurrences of a particular substring:

val.count(",")

2

In [70]:
# replace will substitute occurrences of one pattern for another. 

print(val)

val.replace(",", "::") # replace "," for "::"

a,b,  guido


'a::b::  guido'

In [71]:
# replace is also commonly used to delete patterns too by passing an empty string:

print(val)

val.replace(",", "")

a,b,  guido


'ab  guido'

### Regular Expressions

Regular expressions provide a flexible way to search or match (often more complex)
string patterns in text. A single expression, commonly called a regex, is a string
formed according to the regular expression language. Python’s built-in **re** module is
responsible for applying regular expressions to strings.

The re module functions fall into three categories:
- pattern matching, 
- substitution, and 
- splitting


Table 7-5. Regular expression methods

In [72]:
# Below is a simple example: suppose we want to split a string with a variable number of whitespace
# characters (tabs, spaces, and newlines). When we call re.split(r"\s+", text), the regular expression
# is first compiled, and then its split method is called on the passed text:

import re
text = "foo    bar\t baz  \tqux"
re.split(r"\s+", text)

['foo', 'bar', 'baz', 'qux']

In [73]:
# We can also compile the regex by ourself with re.compile, forming a reusable regex object:

regex = re.compile(r"\s+") # form a regex object
regex.split(text) # apply the split function to the string text

['foo', 'bar', 'baz', 'qux']

In [74]:
#  If we would like to get a list of all patterns matching the regex, you can use the findall method:

regex.findall(text)

['    ', '\t ', '  \t']

In [75]:
# Creating a regex object with re.compile is highly recommended if you intend to apply the same 
# expression to many strings. This will save CPU cycles.

# match and search are closely related to findall. While findall returns all matches
# in a string, search returns only the first match. More rigidly, match only matches at
# the beginning of the string. 

# Let’s consider a block of text and a regular expression capable of identifying most email addresses

# the text is a long string containing the first names and emails of four people
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com"""

# the pattern is about email address: [user name]+@[domain name]+.[domain suffix]
# user name can be a combination of letters, numbers, and special characters ._%+-
# domain name can be a combination of letters, numbers, and special characters .-
# \. means "."
# domain suffix has 2-4 letters
pattern = r"[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}" 


# re.IGNORECASE makes the regex case insensitive

# findall returns all matches (i.e., email addresses) in the string
regex = re.compile(pattern, flags=re.IGNORECASE)

In [76]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [77]:
# search returns a special match object for the first email address in the text

m = regex.search(text)
print(m)

text[m.start():m.end()]

<re.Match object; span=(5, 20), match='dave@google.com'>


'dave@google.com'

In [78]:
# match only matches at the beginning of the string. Since the beginning of the string is 
# not an email address, match returns None

print(regex.match(text))

None


In [79]:
# sub will return a new string with occurrences of the pattern replaced by a new string

print(regex.sub("REDACTED", text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED


In [80]:
# Suppose we would like to find email addresses and simultaneously segment each
# address into its three components: username, domain name, and domain suffix. 

# To do this, put parentheses around the parts of the pattern to segment:
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"
regex = re.compile(pattern, flags=re.IGNORECASE)

# A match object produced by this modified regex returns a tuple of the pattern components
# with its groups method

m = regex.match("wesm@bright.net")
m.groups()

('wesm', 'bright', 'net')

In [81]:
# findall returns a list of tuples when the pattern has groups

regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [82]:
# sub also has access to groups in each match using special symbols like \1 and \2. 
# The symbol \1 corresponds to the first matched group, \2 corresponds to the second, and so forth

print(regex.sub(r"Username: \1, Domain: \2, Suffix: \3", text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com


### String Functions in pandas

Cleaning up a messy dataset for analysis often requires a lot of string manipulation

Table 7-6. Partial listing of Series string methods

In [83]:
# define a dictionary
data = {"Dave": "dave@google.com", "Steve": "steve@gmail.com",
        "Rob": "rob@gmail.com", "Wes": np.nan}

# convert the dictionary to a Series
data = pd.Series(data)
print(data)

# identify missing values
data.isna()

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object


Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [84]:
# Series has array-oriented methods for string operations that skip over and 
# propagate NA values. These are accessed through Series’s str attribute.

# We could check whether each email address has "gmail" in it with str.contains:

data.str.contains("gmail") #  the result indicates that emails of Steve and Rob are gmail

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [85]:
# Note that the result of this operation has an object dtype. pandas has extension types
# that provide for specialized treatment of strings, integers, and Boolean data which
# until recently have had some rough edges when working with missing data

data_as_string_ext = data.astype('string') # we convert data to string
print(data_as_string_ext)
data_as_string_ext.str.contains("gmail") # then we use the str.contains method to return a Boolean Series

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                 <NA>
dtype: string


Dave     False
Steve     True
Rob       True
Wes       <NA>
dtype: boolean

In [86]:
# Regular expressions can be used, too, along with any re options like IGNORECASE

pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [87]:
# There are a couple of ways to do vectorized element retrieval. Either use str.get or
# index into the str attribute:
    
matches = data.str.findall(pattern, flags=re.IGNORECASE).str[0]
print(matches)

matches.str.get(1) # extract the component indexed as 1

Dave     (dave, google, com)
Steve    (steve, gmail, com)
Rob        (rob, gmail, com)
Wes                      NaN
dtype: object


Dave     google
Steve     gmail
Rob       gmail
Wes         NaN
dtype: object

In [88]:
# slice strings using this syntax

data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

In [89]:
# str.extract method will return the captured groups of a regular expression as a DataFrame

data.str.extract(pattern, flags=re.IGNORECASE)

,0,1,2
Dave,dave,google,com
Steve,steve,gmail,com
Rob,rob,gmail,com
Wes,NaN,NaN,NaN


## Categorical Data

This section introduces the pandas Categorical type. We can achieve better performance and memory use in some pandas operations by using it. We will also study some tools that may help with using categorical data in statistics and machine learning applications.

### Background and Motivation

In [90]:
# a column in a table may contain repeated instances of a smaller set of distinct values. 
values = pd.Series(['apple', 'orange', 'apple',
                    'apple'] * 2)
values

# we can use pandas.unique to find the unique values
print('\n unique values are:')
pd.unique(values)

# value_counts gives the frequency
print('\n the frequency distribution:')
values.value_counts()

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object


 unique values are:


array(['apple', 'orange'], dtype=object)


 the frequency distribution:


apple     6
orange    2
Name: count, dtype: int64

In [91]:
# In data warehousing, a best practice is to use so-called dimension tables containing the distinct values
# and storing the primary observations as integer keys referencing the dimension table

# a dimention table saving the distinct values with indices 
dim = pd.Series(['apple', 'orange'])
dim

# a Series saving the instance indices
values = pd.Series([0, 1, 0, 0] * 2)
values


# We can use the take method to restore the original Series of strings:
dim.take(values)

0     apple
1    orange
dtype: object

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

### Categorical Extension Type in pandas

pandas has a special Categorical extension type for holding data that uses the <br>
integer-based categorical representation or encoding. This is a popular data compression <br>
technique for data with many occurrences of similar values and can provide<br>
significantly faster performance with lower memory use, especially for string data

In [92]:
# Consider the following example, 

structures = ['concrete', 'steel', 'concrete', 'steel'] * 2
N = len(structures)
rng = np.random.default_rng(seed=12345)
df = pd.DataFrame({'structure': structures,
                   'structure_id': np.arange(N),
                   'count': rng.integers(3, 15, size=N), # generate N random integers from the discrete uniform distribution ranging from 3(inclusive) to 15(exclusive) 
                   'age': rng.uniform(0, 20, size=N)}, # generate N random numbers from the continuous uniform distribution ranging from 0(inclusive) to 204(exclusive)
                  columns=['structure', 'structure_id', 'count', 'age'])
df

,structure,structure_id,count,age
0,concrete,0,11,7.822191
1,steel,1,5,6.656279
2,concrete,2,12,11.966175
3,steel,3,6,3.734684
4,concrete,4,5,13.455121
5,steel,5,12,18.836057
6,concrete,6,10,4.964914
7,steel,7,11,18.977623


In [93]:
# df['structure'] is an array of Python string objects. We can convert it to categorical 
# by calling astype('category'):
structure_cat = df['structure'].astype('category')
structure_cat 

#  The values for structure_cat  are now an instance of pandas.Categorical, 
# which can be accessed via the .array attribute
c=structure_cat.array

# The Categorical object has an attribute called categories, which returns distinct categories in the object
c.categories

# The Categorical object has an attribute called code, which returns the encoding representations of the object
c.codes

# A useful trick to get a mapping between integer-based codes and categories is 
dict(enumerate(c.categories))

0    concrete
1       steel
2    concrete
3       steel
4    concrete
5       steel
6    concrete
7       steel
Name: structure, dtype: category
Categories (2, object): ['concrete', 'steel']

Index(['concrete', 'steel'], dtype='object')

array([0, 1, 0, 1, 0, 1, 0, 1], dtype=int8)

{0: 'concrete', 1: 'steel'}

In [94]:
# You can convert a DataFrame column to categorical by assigning the converted result

df['structure'] = df['structure'].astype('category')
df["structure"]

0    concrete
1       steel
2    concrete
3       steel
4    concrete
5       steel
6    concrete
7       steel
Name: structure, dtype: category
Categories (2, object): ['concrete', 'steel']

In [95]:
# we can also create pandas.Categorical directly from other types of Python sequences:

my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

In [96]:
# categorical encoded data 
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]

# use pandas.Categorical.from_codes(codes, categories) to construct the Categorical object
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [97]:
# Unless explicitly specified, categorical conversions assume no specific ordering of the
# categories. So the categories array may be in a different order depending on the
# ordering of the input data. When using from_codes or any of the other constructors,
# we can indicate that the categories have a meaningful ordering

ordered_cat = pd.Categorical.from_codes(codes, categories,
                                        ordered=True)
ordered_cat

# The output [foo < bar < baz] indicates that 'foo' precedes 'bar' in the ordering, and so on.

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

In [98]:
# An unordered categorical instance can be made ordered with as_ordered:
my_cats_2

my_cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

### Computations with Categoricals

Using Categorical in pandas compared with the nonencoded version (like an array
of strings) generally behaves the same way. Some parts of pandas, like the groupby
function, perform better when working with categoricals. There are also some functions
that can utilize the ordered flag

In [99]:
# generate an array of size 1000 from standard normal distribution

rng = np.random.default_rng(seed=12345)
draws = rng.standard_normal(1000)
draws[:5]

array([-1.4238,  1.2637, -0.8707, -0.2592, -0.0753])

In [100]:
# the pandas.qcut binning function returns a pandas Categorical object
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins

['Q1', 'Q4', 'Q1', 'Q2', 'Q2', ..., 'Q3', 'Q3', 'Q2', 'Q3', 'Q2']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [101]:
# so we can use groupby to extract some summary statistics
results = (pd.Series(draws)
           .groupby(pd.Series(bins, name='quartile'),
                   observed=False)
           .agg(['count', 'min', 'max'])
           .reset_index())
results

# The 'quartile' column in the result retains the original categorical information, 
# including ordering, from bins
results['quartile']

,quartile,count,min,max
0,Q1,250,-3.119609,-0.678494
1,Q2,250,-0.673305,0.008009
2,Q3,250,0.018753,0.686183
3,Q4,250,0.688282,3.211418


0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [102]:
# Better performance with categoricals

# In this example, we consider some Series with 10 million elements
# but a small number of distinct categories
N = 10_000_000
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4)) # N//4 is floor division
# memory usage of pandas series in categorical. 
labels.memory_usage(deep=True)

# memory usage of pandas series in categorical. 
categories = labels.astype('category')
categories.memory_usage(deep=True)

# The comparison shows that the categorical object uses significantly less memory than string

# The conversion to category is not free, of course, but it is a one-time cost
%time _ = labels.astype('category')

# GroupBy operations can be significantly faster with categoricals because the underlying
# algorithms use the integer-based codes array instead of an array of strings
# we compare the performance of value_counts(), which internally uses the GroupBy machinery:
%timeit labels.value_counts()
%timeit categories.value_counts()

600000128

10000540

CPU times: user 173 ms, sys: 52.9 ms, total: 226 ms
Wall time: 226 ms
181 ms ± 502 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
19.7 ms ± 80.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Categorical Methods

< This can be skipped > <br>
Table 7-7. Categorical methods for Series in pandas

In [103]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [104]:
# The special accessor attribute cat provides access to categorical methods

# access categoies
cat_s.cat.categories

# access codes 
cat_s.cat.codes

Index(['a', 'b', 'c', 'd'], dtype='object')

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [105]:
# Suppose that we know the actual set of categories for this data extends beyond the four values 
# observed in the data. We can use the set_categories method to change them:

actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

# While it appears that the data is unchanged, the new categories will be reflected
# in operations that use them
cat_s.value_counts() # # cat_s has four categories
cat_s2.value_counts() # cat_s2 now has five categories

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

a    2
b    2
c    2
d    2
Name: count, dtype: int64

a    2
b    2
c    2
d    2
e    0
Name: count, dtype: int64

In [106]:
# After we filter a large DataFrame or Series, many of the categories may not appear in the data. 
# To help with this, we can use the remove_unused_categories method to trim unobserved categories:

cat_s

cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3

cat_s3.cat.remove_unused_categories()
# 'c' and 'd' are unused categories that are removed

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']

Creating dummy variables for modeling

When we use statistics or machine learning tools, we often transform categorical
data into dummy variables, also known as one-hot encoding. This involves creating
a DataFrame with a column for each distinct category; these columns contain 1s
for occurrences of a given category and 0 otherwise.